In [38]:
import numpy as np
import pandas as pd
from scipy import optimize

from gaus_vertex import *

In [2]:
# posible alternative to dataframes: write to csv directly. not necessary bc find_phase() is the bottleneck
# from io import StringIO
# from csv import writer 

In [34]:
%%time
# faster! 
# for n_mixs data points, generates an array with colomns for each monomer in mix, 
# polymer relative volume fractions, chi value, and phase identification

n_mixs = 100
M = 2 # number of monomers per polymer
N_m = 1000# length of monomer
b = 1# kuhn length
n_p = 10# number of polymer types per mix
chi_array = np.arange(8, 25, 1)/(N_m*M)

# generate polymers and densities
rand_polys = np.random.choice(2,(n_mixs, n_p, M))
rand_dens = np.random.dirichlet(np.ones(n_p),size=(1, n_mixs))[0]

# assign phase to each mix
data = pd.DataFrame(np.zeros((n_mixs*len(chi_array),M*n_p+n_p+2)))

for i, mix in enumerate(rand_polys):
    if not (np.all(mix == np.array([0]*M)) or np.all(mix == np.array([1]*M))): #in case all homopolymers (meaning no separation at all)
        dens = rand_dens[i]
        
        res = find_phase(mix, dens, N_m, b, M, chi_array) #returns an array of phase results at a range of chi values
        chis = res[::2]
        phase_names = res[1::2]

        polys = np.append(mix.flatten(), dens)
        polys = np.vstack([polys]*len(chis)) #duplicate polys, since the mix is the same for the set of chis
        
        rows = np.column_stack((polys, chis, phase_names))
        data.iloc[i*len(chis):i*len(chis)+len(chis)] = rows
    else:
        i-=1

path = r"C:\Users\Thomas\Desktop\structure_factor\structure_factor" + "\phase_data_M=" + str(M) + "_1e" + str(np.log10(n_mixs)) + ".csv"
data.to_csv(path, index=False, header=False)

Wall time: 11 s


In [37]:
10*10000/(60*60*24)

1.1574074074074074

In [ ]:
# transition to towers (JUST THE DATA-CREATION code)
# time trials to see if old tower work( if i use like 5, how quckly)
# RUN!! in parallel

In [7]:
# %%time
# #WORKS BUT SLOW
# # for n_mixs data points, generates an array with colomns for each monomer in mix, 
# # polymer relative volume fractions, chi value, and phase identification

# n_mixs = 100
# M = 2 # number of monomers per polymer
# N_m = 1# length of monomer
# b = 1# kuhn length
# n_p = 10# number of polymer types per mix
# chi_array = np.arange(8, 25, 1)/(N_m*M)

# # generate polymers and densities
# rand_polys = np.random.choice(2,(n_mixs, n_p, M))
# rand_dens = np.random.dirichlet(np.ones(n_p),size=(1, n_mixs))[0]

# # assign phase to each mix
# data = pd.DataFrame()

# for i, mix in enumerate(rand_polys):
#     if not np.all(mix == mix[0]): #in case all homopolymers (meaning no separation at all)
#         dens = rand_dens[i]
        
#         res = find_phase(mix, dens, N_m, b, M, chi_array) #returns an array of phase results at a range of chi values
#         chis = res[::2]
#         phase_names = res[1::2]
#         for i in range(len(chis)):
# #             row = np.append(mix.flatten(), [dens.flatten(), float(chis[i]), phase_names[i]])
#             row = np.append(np.append(mix.flatten(), dens), [float(chis[i]), phase_names[i]])
#             row = pd.DataFrame(row).T
            
#             if len(data) == 0:
#                 data = row
#             else:
#                 data = data.append(row)

# # path = r"C:\Users\Thomas\Desktop\structure_factor\structure_factor" + "\phase_data_M=" + str(M) + "_1e" + str(np.log10(n_mixs)) + ".csv"
# # data.to_csv(path, index=False, header=False)

Wall time: 12.3 s


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

#split X (polys connectivity and densities) and Y (micro or macro)
X = data.iloc[:,0:-1]
y = data.iloc[:,-1]

#train test split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1, test_size=0.2)

#generate MLP
clf = MLPClassifier(hidden_layer_sizes=(32,32),activation="relu",random_state=1).fit(X_train, y_train)

train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)

#         acc_array = np.append(acc_array, np.array([M, n_mixs, train_score, test_score]))
acc_array=np.array([])
if len(acc_array) == 0:
    acc_array = np.array([M, n_mixs, train_score, test_score])
else:
    acc_array = np.vstack([acc_array, np.array([M, n_mixs, train_score, test_score])])



In [ ]:
acc_array